# 납기 지연 여부 예측 문제
- **Main Task : 납기 지연 여부 예측 성능 높이기 (베이스라인 점수: 0.76664)**
- **Sub Task : 각종 시각화를 통해 재미있는 정보 확인해보기**

# 초기 설정

### 라이브러리 불러오기

In [ ]:
import pandas as pd  # 판다스 : 데이터 조작을 위한 라이브러리
import numpy as np  # 넘파이 : 계산을 위한 라이브러리
import matplotlib as mpl  # 맫플롯립 : 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns  # 씨본 : 시각화 라이브러리

pd.set_option('display.max_columns', None)  # DataFrame의 열을 모두 출력하도록 설정

### 데이터 불러오기

In [ ]:
data_path = '/kaggle/input/dataco-global-supply-chain/'  # 데이터 경로

train = pd.read_csv(data_path + 'train.csv')  # 훈련 데이터
test = pd.read_csv(data_path + 'test.csv')  # 테스트 데이터
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')  # 제출 샘플 데이터

# 탐색적 데이터 분석(EDA)

### 데이터 둘러보기

In [ ]:
train.head()

In [ ]:
# 변수명, 비결측값 개수, 데이터 타입 출력
train.info()

### 데이터 시각화

#### 타깃값 (Late Delivery)

In [ ]:
mpl.rc('font', size=10)  # 글자 폰트 크기 설정
plt.figure(figsize=(6, 5))  # 그래프 크기 설정

sns.countplot(x='Late Delivery', data=train)  # 카운트 플롯 그리기
plt.title('Target Distribution');  # 그래프 제목 설정

- Late Delivery 1(납기 지연) 건수가 Late Delivery 0(납기 준수) 건수보다 많음

#### Benefit per order
- `Benefit per order` 전체를 활용한 박스플롯

In [ ]:
np.min(train["Benefit per order"])

In [ ]:
sns.boxplot(y='Benefit per order', data=train)
plt.title('Boxplot of "Benefit per order"');

- 이상치를 제외한 `Benefit per order`의 박스플롯

In [ ]:
Q1 = train['Benefit per order'].quantile(0.25)  # 하위 25%
Q3 = train['Benefit per order'].quantile(0.75)  # 상위 25%
IQR = Q3 - Q1
upper_limit = Q3 + 1.5*(IQR)
lower_limit = Q1 - 1.5*(IQR)

print(upper_limit, lower_limit)

In [ ]:
train_copy = train.copy()  # 원본 데이터 복사
train_copy = train_copy.loc[(lower_limit < train_copy['Benefit per order']) & (train_copy['Benefit per order'] < upper_limit)]  # 이상치 제거

sns.boxplot(y='Benefit per order', data=train_copy)
plt.title('Boxplot of "Benefit per order without outliers"');

- `Benefit per order`의 분포도

In [ ]:
sns.displot(train_copy['Benefit per order'])
plt.title('Distribution of Benefit per order without outliers');

In [ ]:
sns.violinplot(y='Benefit per order', data=train_copy)
plt.title('Violinplot of Benefit per order without outliers');

### <font color='red'>★ 다른 변수들도 활용해 박스플롯, displot, 바이올린플롯을 그려보시고, 그 변수들의 패턴이나 경향이 어떠한지 파악해보세요.</font>

# 저는 트리 모델이 이상치에 덜 민감하기도 하고, sales 같은 값들은 원래 천차만별이라는 생각으로 이상치 처리를 따로 하지 않았습니다.

In [ ]:
sns.boxplot(y='Sales per customer', data=train)
plt.title('Boxplot of "Benefit per order"');

In [ ]:
Q1 = train_copy['Sales per customer'].quantile(0.25)  # 하위 25%
Q3 = train_copy['Sales per customer'].quantile(0.75)  # 상위 25%
IQR = Q3 - Q1
upper_limit = Q3 + 1.5*(IQR)
lower_limit = Q1 - 1.5*(IQR)

train_copy = train_copy.loc[(lower_limit < train_copy['Sales per customer']) & (train_copy['Sales per customer'] < upper_limit)]  # 이상치 제거

sns.boxplot(y='Sales per customer', data=train_copy)
plt.title('Boxplot of "Benefit per order without outliers"');

In [ ]:
sns.boxplot(y='Sales per customer', data=train)
plt.title('Boxplot of "Benefit per order"');

In [ ]:
Q1 = train_copy['Sales'].quantile(0.25)  # 하위 25%
Q3 = train_copy['Sales'].quantile(0.75)  # 상위 25%
IQR = Q3 - Q1
upper_limit = Q3 + 1.5*(IQR)
lower_limit = Q1 - 1.5*(IQR)

train_copy = train_copy.loc[(lower_limit < train_copy['Sales']) & (train_copy['Sales'] < upper_limit)]  # 이상치 제거

sns.boxplot(y='Sales', data=train_copy)
plt.title('Boxplot of "Benefit per order without outliers"');

#### Customer State
- `Customer State`의 카운트플롯

In [ ]:
plt.figure(figsize=(12, 4))

sns.countplot(x='Sales', data=train_copy)
plt.xticks(rotation=90)  # x축 레이블 90도 회전
plt.title('Countplot of Customer State');

### <font color='red'>★ 다른 카테고리 변수들도 활용해 카운트플롯을 그려보시고, 어떤 값이 많은지 살펴보세요.</font>

#### (optional) Category Name
- `Category Name`의 카운트플롯 (상위 20개만)

In [ ]:
plt.figure(figsize=(12, 4))

category_name_value_counts = train_copy['Type'].value_counts()
selected_category_name = list(category_name_value_counts.index[:20])  # 상위 20개만 추출

sns.countplot(x='Type', data=train_copy, order=selected_category_name, palette='Set2')
plt.xticks(rotation=90)  # x축 레이블 90도 회전
plt.title('Categories by Count');

#### (optional) Market

- `Market`의 파이그래프

In [ ]:
unique_markets = list(train_copy['Market'].unique())  # Market 변수에 들어있는 고윳값
markets = list(train_copy['Market'])
market_counts = [markets.count(market) for market in unique_markets]  # Market 변수에 들어있는 고윳값별 개수

# 파이그래프 그리기
plt.pie(market_counts, labels=unique_markets, autopct='%1.1f%%');

### Category Name vs. Late Delivery

- `Category Name`별 납기 지연 비율

In [ ]:
plt.figure(figsize=(20, 5))
sns.barplot(x='Customer Segment', y='Late Delivery', data=train_copy)
plt.xticks(rotation=90)  # x축 레이블 90도 회전
plt.title('Late Delivery Rate by Category Name');

### <font color='red'>★ 다른 카테고리 변수들도 활용해 카운트플롯을 그려보시고, 변수 내 고윳값별 타깃값 비율(납지 지연 비율)이 어떻게 다른지 경향을 파악해보세요.</font>

### (optional) Customer ID vs. Late Delivery

- `고객 ID`별 납기 지연 비율

In [ ]:
plt.figure(figsize=(12, 4))

customer_id_value_counts = train['Order Profit Per Order'].value_counts()
selected_customer_id = list(customer_id_value_counts.index[:20])
sns.barplot(x='Order Profit Per Order', y='Late Delivery', data=train, order=selected_customer_id)

plt.xticks(rotation=90)
plt.title('Late Delivery Rate by Customer Id');

### 상관관계

In [ ]:
mpl.rc('font', size=7)  # 글자 폰트 크기 설정
plt.figure(figsize=(20, 10))

cont_corr = train.corr()
sns.heatmap(cont_corr, annot=True, cmap='OrRd');

# 피처 엔지니어링

### [파생 변수 생성] 주문 연도, 주문 월 변수 추가

- 어떤 작업을 하려면 훈련 데이터와 테스트 데이터에 동일하게 해줘야 합니다.

In [ ]:
train['order date (DateOrders)']

In [ ]:
train['order date (DateOrders)'] = pd.to_datetime(train['order date (DateOrders)'])
test['order date (DateOrders)'] = pd.to_datetime(test['order date (DateOrders)'])
#shipping date (DateOrders)(미정)-orderdate
train['shipping date (DateOrders)'] = pd.to_datetime(train['shipping date (DateOrders)'])
test['shipping date (DateOrders)'] = pd.to_datetime(test['shipping date (DateOrders)'])

train['diff'] =  train['shipping date (DateOrders)'] - train['order date (DateOrders)']
test['diff'] = test['shipping date (DateOrders)'] - test['order date (DateOrders)']

train['Order Year'] = train['order date (DateOrders)'].dt.year    
train['Order Month'] = train['order date (DateOrders)'].dt.month 

test['Order Year'] = test['order date (DateOrders)'].dt.year    
test['Order Month'] = test['order date (DateOrders)'].dt.month 

train

In [ ]:
train['diff'] = train['diff'].astype('object')
test['diff'] = test['diff'].astype('object')

train['shipping date (DateOrders)'] =train['shipping date (DateOrders)'].astype('object')
test['shipping date (DateOrders)'] = test['shipping date (DateOrders)'].astype('object')

### 필요 없는 변수 제거
### <font color='red'>★ 필요 없는 변수로는 뭐가 더 있을지 고민해보시고(직관 + 시각화 결과 활용), 아래 drop_features 리스트에 추가해보세요. (생각보다 필요 없는 변수가 많습니다.)</font>

In [ ]:
drop_features = ['Customer Email', 'Customer Password', 'Product Image', 'order date (DateOrders)', 'Product Description', 'Order Id', 'Department Name', 'Order Zipcode']
train = train.drop(drop_features, axis=1)
test = test.drop(drop_features, axis=1)

In [ ]:
train

### 인코딩
- train.info()를 출력할 때 object 타입을 나타내는 변수는 모두 문자로 된 데이터입니다.
- 문자로 된 데이터는 숫자로 바꾸어주어야 합니다(이를 인코딩이라고 합니다.)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# 인코딩이란 문자열 데이터를 숫자형 데이터로 변환하는 작업을 뜻함 (기본적으로 문자열 타입(object type)은 모두 인코딩해야 합니다.)
# 인코딩할 변수 지정 
features_to_encode = train.select_dtypes(include=['object']).columns

# 인코더 생성 (훈련 데이터에서 보지 못한 값이 테스트 데이터에 등장하면 -1로 처리)
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터에 인코딩 피팅
ordinal_encoder.fit(train[features_to_encode])

# 훈련 데이터와 테스트 데이터에 인코딩 적용
train[features_to_encode] = ordinal_encoder.transform(train[features_to_encode])
test[features_to_encode] = ordinal_encoder.transform(test[features_to_encode])

### 훈련 데이터, 테스트 데이터, 검증 데이터 나누기

In [ ]:
# 타깃값
y_train = train['Late Delivery'] 

# 타깃값 count 제거
X_train = train.drop(['Late Delivery'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 나누기 (8대 2 비율)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 모델링

### 모델 훈련
### <font color='red'>★★★ learning_rate나 n_estimators 등의 변수를 바꾸면서 결과가 어떻게 달라지는지 테스트해보세요!</font>
- https://lightgbm.readthedocs.io/en/latest/Parameters.html 에서 LightGBM에 들어갈 모든 파라미터를 확인할 수 있습니다. 정말 많죠? 다 아실 필요 없습니다. 10가지 미만의 주요 파라미터 정도만 나중에 숙지하셔도 충분합니다.

In [ ]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터 / 기본적으로는 learning_rate를 줄이면서, n_estimators를 늘리는 방향으로 해보세요!
params = {'num_leaves': 255,
          'objective': 'binary',
          'learning_rate': 0.05,
          'n_estimators': 300,
          'random_state': 98}

# LightGBM 분류 모델
model = lgb.LGBMClassifier(**params)

# 모델 훈련
model.fit(X_train,  # 훈련 데이터의 변수
          y_train,  # 훈련 데이터의 타깃값
          eval_set=[(X_valid, y_valid)],  # 모델 검증에 사용할 검증 데이터
          callbacks=[lgb.log_evaluation(100)])  # 매 100번의 이터레이션마다 검증 데이터 평가 점수 출력

### 결과 예측

In [ ]:
preds = model.predict(test)

In [ ]:
preds

# 제출

In [ ]:
sample_submission['Late Delivery'] = preds.astype(int)   # 예측값을 정수값으로 변환
sample_submission.to_csv('submission.csv', index=False)  # 예측값이 포함된 sample_submission을 최종 파일로 제출 

In [ ]:
# 예측값이 포함된 제출 파일 확인
submission = sample_submission
submission